In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

Este proceso de ETL se realiza con la finalidad de verificar la estructura de los datos. En ciertos archivos, existen múltiples saltos de linea inusuales que entorpecen la lectura de la librería de *Pandas*. Además pueden haber el mismo caracter que separa las columnas dentro de los textos y esto entorpece la lectura de *Power BI*, por lo que se cargará con un caracter único en toda la data.

# Extract

In [2]:
coursera_courses = pd.read_csv('_src/data_lake/Moocs/Coursera_courses.csv')
coursera_reviews = pd.read_csv('_src/data_lake/Moocs/Coursera_reviews.csv')
edx_courses = pd.read_csv('_src/data_lake/Moocs/edx_courses.csv')
udemy_courses = pd.read_csv('_src/data_lake/Moocs/udemy_courses.csv')

# Transform

In [3]:
def replace_newlines(df : pd.DataFrame, value = '. '):
    for i in range(5):
        for row in df.columns:
            df[row] = df[row].apply(lambda x: str(x).replace('\\n', '\n'))
            df[row] = df[row].apply(lambda x: str(x).replace('\\r', '\r'))

    for row in df.columns:
            df[row] = df[row].apply(lambda x: str(x).replace('\r', value).replace('\n', value))
    edx_courses.replace('nan',np.NaN, inplace=True)
    

### Coursera_courses.csv

In [4]:
coursera_courses.head()

,name,institution,course_url,course_id
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning
1,Indigenous Canada,University of Alberta,https://www.coursera.org/learn/indigenous-canada,indigenous-canada
2,The Science of Well-Being,Yale University,https://www.coursera.org/learn/the-science-of-...,the-science-of-well-being
3,Technical Support Fundamentals,Google,https://www.coursera.org/learn/technical-suppo...,technical-support-fundamentals
4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,https://www.coursera.org/learn/google-cbrs-cpi...,google-cbrs-cpi-training


In [5]:
coursera_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         623 non-null    object
 1   institution  623 non-null    object
 2   course_url   623 non-null    object
 3   course_id    623 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB


In [6]:
coursera_courses.isna().sum()

name           0
institution    0
course_url     0
course_id      0
dtype: int64

Valores Duplicados

In [7]:
coursera_courses[coursera_courses.duplicated(keep=False)].sort_values('course_id')

,name,institution,course_url,course_id


Todo parece correcto, ahora toca verificar los separadores de columnas.

la siguiente selda selecciona todos los datos string y busca un caracter específico.

In [8]:
coursera_courses.select_dtypes([object]).apply(lambda x: x.str.contains(',').sum())

name           38
institution    37
course_url      0
course_id       0
dtype: int64

Vemos que no es recomendable usar comas ',' ya que existe ese caracter en dos columnas.

In [9]:
coursera_courses.select_dtypes([object]).apply(lambda x: x.str.contains('¦').sum())

name           0
institution    0
course_url     0
course_id      0
dtype: int64

No existen saltos de linea dentro de las columnas

In [10]:
coursera_courses.select_dtypes([object]).apply(lambda x: x.str.contains('\n').sum())

name           0
institution    0
course_url     0
course_id      0
dtype: int64

Eliminamos los registros exactamente iguales en todos los campos.

In [11]:
coursera_courses.drop_duplicates(inplace=True)

### coursera_reviews.csv

In [12]:
coursera_reviews.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


In [13]:
coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   reviews       1454571 non-null  object
 1   reviewers     1454711 non-null  object
 2   date_reviews  1454711 non-null  object
 3   rating        1454711 non-null  int64 
 4   course_id     1454711 non-null  object
dtypes: int64(1), object(4)
memory usage: 55.5+ MB


In [14]:
coursera_reviews.isna().sum()

reviews         140
reviewers         0
date_reviews      0
rating            0
course_id         0
dtype: int64

In [15]:
coursera_reviews[coursera_reviews.duplicated(keep=False)].sort_values('reviews')

,reviews,reviewers,date_reviews,rating,course_id
842121,"\tAs a general feedback, first I would ask you...",By Carlos O,"Sep 15, 2020",5,astro
842171,"\tAs a general feedback, first I would ask you...",By Carlos O,"Sep 15, 2020",5,astro
842146,"\tAs a general feedback, first I would ask you...",By Carlos O,"Sep 15, 2020",5,astro
1238309,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAn\nexcell...,By Alan K,"May 31, 2020",5,systems-engineering
1238334,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAn\nexcell...,By Alan K,"May 31, 2020",5,systems-engineering
...,...,...,...,...,...
1422276,NaN,By Ritu K,"Aug 04, 2020",4,technical-support-fundamentals
1454669,NaN,By Christopher L J,"Sep 26, 2020",5,computer-networking
1454678,NaN,By Jerold K G,"Jun 10, 2020",5,computer-networking
1454691,NaN,By Christopher L J,"Sep 26, 2020",5,computer-networking


In [16]:
coursera_reviews.select_dtypes([object]).replace(np.NaN,'').apply(lambda x: x.str.contains('\n').sum())

reviews         13426
reviewers           0
date_reviews        0
course_id           0
dtype: int64

In [17]:
replace_newlines(coursera_reviews)

In [18]:
coursera_reviews.select_dtypes([object]).replace(np.NaN,'').apply(lambda x: x.str.contains('¦').sum())

reviews         6
reviewers       0
date_reviews    0
rating          0
course_id       0
dtype: int64

In [19]:
coursera_reviews.reviews = coursera_reviews.reviews.str.replace('¦', '|')

In [20]:
coursera_reviews['date_reviews'] = pd.to_datetime(coursera_reviews['date_reviews'], format='%b %d, %Y', errors='coerce')

### edx_courses

In [21]:
edx_courses.head()

,title,summary,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,course_effort,course_length,price,course_description,course_syllabus,course_url
0,How to Learn Online,Learn essential strategies for successful onli...,"124,980",Self-paced on your time,edX,Nina Huntemann-Robyn Belair-Ben Piscopo,Introductory,Education & Teacher Training,English,English,2–3 hours per week,2 Weeks,FREE-Add a Verified Certificate for $49 USD,"Designed for those who are new to elearning, t...",Welcome - We start with opportunities to meet ...,https://www.edx.org/course/how-to-learn-online
1,Programming for Everybody (Getting Started wit...,"This course is a ""no prerequisite"" introductio...","293,864",Self-paced on your time,The University of Michigan,Charles Severance,Introductory,Computer Science,English,English,2–4 hours per week,7 Weeks,FREE-Add a Verified Certificate for $49 USD,This course aims to teach everyone the basics ...,NaN,https://www.edx.org/course/programming-for-eve...
2,CS50's Introduction to Computer Science,An introduction to the intellectual enterprise...,"2,442,271",Self-paced on your time,Harvard University,David J. Malan-Doug Lloyd-Brian Yu,Introductory,Computer Science,English,English,6–18 hours per week,12 Weeks,FREE-Add a Verified Certificate for $90 USD,"This is CS50x , Harvard University's introduct...",NaN,https://www.edx.org/course/cs50s-introduction-...
3,The Analytics Edge,"Through inspiring examples and stories, discov...","129,555",Instructor-led on a course schedule,Massachusetts Institute of Technology,Dimitris Bertsimas-Allison O'Hair-John Silberh...,Intermediate,Data Analysis & Statistics,English,English,10–15 hours per week,13 Weeks,FREE-Add a Verified Certificate for $199 USD,"In the last decade, the amount of data availab...",NaN,https://www.edx.org/course/the-analytics-edge
4,Marketing Analytics: Marketing Measurement Str...,This course is part of a MicroMasters® Program,"81,140",Self-paced on your time,"University of California, Berkeley",Stephan Sorger,Introductory,Computer Science,English,English,5–7 hours per week,4 Weeks,FREE-Add a Verified Certificate for $249 USD,Begin your journey in a new career in marketin...,NaN,https://www.edx.org/course/marketing-analytics...


In [22]:
edx_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               975 non-null    object
 1   summary             922 non-null    object
 2   n_enrolled          855 non-null    object
 3   course_type         975 non-null    object
 4   institution         975 non-null    object
 5   instructors         972 non-null    object
 6   Level               975 non-null    object
 7   subject             975 non-null    object
 8   language            975 non-null    object
 9   subtitles           972 non-null    object
 10  course_effort       975 non-null    object
 11  course_length       975 non-null    object
 12  price               975 non-null    object
 13  course_description  935 non-null    object
 14  course_syllabus     417 non-null    object
 15  course_url          975 non-null    object
dtypes: object(16)
memory usage

In [23]:
edx_courses.isna().sum()

title                   0
summary                53
n_enrolled            120
course_type             0
institution             0
instructors             3
Level                   0
subject                 0
language                0
subtitles               3
course_effort           0
course_length           0
price                   0
course_description     40
course_syllabus       558
course_url              0
dtype: int64

Valores Duplicados

In [24]:
edx_courses[edx_courses.duplicated(keep=False)].sort_values('title')

,title,summary,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,course_effort,course_length,price,course_description,course_syllabus,course_url
59,Introduction to Probability,"Learn probability, an essential language and s...","97,291",Self-paced on your time,Harvard University,Joseph Blitzstein,Intermediate,Data Analysis & Statistics,English,English,5–10 hours per week,10 Weeks,FREE-Add a Verified Certificate for $99 USD,Probability and statistics help to bring logic...,NaN,https://www.edx.org/course/introduction-to-pro...
95,Introduction to Probability,"Learn probability, an essential language and s...","97,291",Self-paced on your time,Harvard University,Joseph Blitzstein,Intermediate,Data Analysis & Statistics,English,English,5–10 hours per week,10 Weeks,FREE-Add a Verified Certificate for $99 USD,Probability and statistics help to bring logic...,NaN,https://www.edx.org/course/introduction-to-pro...


In [25]:
edx_courses.select_dtypes([object]).replace(np.NaN,'').apply(lambda x: x.str.contains('\n').sum())

title                   0
summary                 2
n_enrolled              0
course_type             0
institution             0
instructors             0
Level                   0
subject                 0
language                0
subtitles               0
course_effort           0
course_length           0
price                   0
course_description     46
course_syllabus       162
course_url              0
dtype: int64

In [26]:
replace_newlines(edx_courses)

In [27]:
edx_courses.select_dtypes([object]).replace(np.NaN,'').apply(lambda x: x.str.contains('¦').sum())

title                 0
summary               0
n_enrolled            0
course_type           0
institution           0
instructors           0
Level                 0
subject               0
language              0
subtitles             0
course_effort         0
course_length         0
price                 0
course_description    0
course_syllabus       0
course_url            0
dtype: int64

Todos los cursos de edx en este dataset son gratuitos, aunque si quieres una certificación debes pagar.

In [28]:
import re
edx_courses.price.apply(lambda x: re.sub(r'[0-9]+', '', x)).unique()

array(['FREE-Add a Verified Certificate for $ USD',
       'FREE-Add a Verified Certificate for $. USD'], dtype=object)

Así que se opta por dejar sólo el precio de verificación, obviando el hecho de que son cursos gratuitos.

In [29]:
edx_courses.price = edx_courses.price.apply(lambda x: re.sub(r'[a-z]+|\$| |-', '', x.lower()))

In [30]:
edx_courses.rename(columns={'price':'certificate_price'}, inplace=True)

la columna *n_enrolled* es una columna de enteros que separa los miles por comas ','. Esto dificulta la lectura de las tecnologías que se utilizan por lo que se eliminarán.

In [31]:
edx_courses.n_enrolled = edx_courses.n_enrolled.str.replace(',','')

In [32]:
edx_courses.course_effort.unique()

array(['2–3 hours per week', '2–4 hours per week', '6–18 hours per week',
       '10–15 hours per week', '5–7 hours per week',
       '8–10 hours per week', '1–3 hours per week', '3–4 hours per week',
       '3–5 hours per week', '2–6 hours per week', '1–2 hours per week',
       '2–5 hours per week', '4–6 hours per week', '10–30 hours per week',
       '6–9 hours per week', '3–6 hours per week', '5–10 hours per week',
       '4–5 hours per week', '5–8 hours per week', '5–6 hours per week',
       '9–10 hours per week', '4–8 hours per week',
       '15–20 hours per week', '6–8 hours per week',
       '10–14 hours per week', '10–20 hours per week',
       '8–12 hours per week', '4–10 hours per week',
       '10–12 hours per week', '7–10 hours per week',
       '3–7 hours per week', '1–4 hours per week', '6–10 hours per week',
       '1–5 hours per week', '8–9 hours per week', '6–12 hours per week',
       '3–8 hours per week', '1–10 hours per week',
       '10–18 hours per week', '4–12 

Dado que todas los valores de course_effort están expresadas en horas por semana, se va a obviar en los valores

In [33]:
edx_courses.course_effort = edx_courses.course_effort.apply(lambda x: re.sub(r'[a-z]| ', '', x.lower()))

In [34]:
edx_courses.rename(columns={'course_effort':'course_effort_hs_per_week'}, inplace=True)

In [35]:
edx_courses.course_length = edx_courses.course_length.apply(lambda x: re.sub(r'[a-z]| ', '', x.lower()))

In [36]:
edx_courses.course_length.unique()

array(['2', '7', '12', '13', '4', '6', '10', '8', '5', '16', '15', '1',
       '11', '14', '9', '3', '18', '17'], dtype=object)

In [37]:
edx_courses.rename(columns={'course_length':'course_length_weeks'}, inplace=True)

### udemy_courses

In [38]:
udemy_courses.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance


In [39]:
udemy_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_id            3678 non-null   int64  
 1   course_title         3678 non-null   object 
 2   url                  3678 non-null   object 
 3   is_paid              3678 non-null   bool   
 4   price                3678 non-null   int64  
 5   num_subscribers      3678 non-null   int64  
 6   num_reviews          3678 non-null   int64  
 7   num_lectures         3678 non-null   int64  
 8   level                3678 non-null   object 
 9   content_duration     3678 non-null   float64
 10  published_timestamp  3678 non-null   object 
 11  subject              3678 non-null   object 
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 319.8+ KB


In [40]:
udemy_courses.isna().sum()

course_id              0
course_title           0
url                    0
is_paid                0
price                  0
num_subscribers        0
num_reviews            0
num_lectures           0
level                  0
content_duration       0
published_timestamp    0
subject                0
dtype: int64

Valores Duplicados

In [41]:
udemy_courses[udemy_courses.duplicated(keep=False)].sort_values('course_id')

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
2478,28295,Learn Web Designing & HTML5/CSS3 Essentials in...,https://www.udemy.com/build-beautiful-html5-we...,True,75,43285,525,24,All Levels,4.000000,2013-01-03T00:55:31Z,Web Development
2561,28295,Learn Web Designing & HTML5/CSS3 Essentials in...,https://www.udemy.com/build-beautiful-html5-we...,True,75,43285,525,24,All Levels,4.000000,2013-01-03T00:55:31Z,Web Development
1234,185526,MicroStation - Células,https://www.udemy.com/microstation-celulas/,True,20,0,0,9,Beginner Level,0.616667,2014-04-15T21:48:55Z,Graphic Design
1473,185526,MicroStation - Células,https://www.udemy.com/microstation-celulas/,True,20,0,0,9,Beginner Level,0.616667,2014-04-15T21:48:55Z,Graphic Design
453,837322,Essentials of money value: Get a financial Life !,https://www.udemy.com/essentials-of-money-value/,True,20,0,0,20,All Levels,0.616667,2016-05-16T18:28:30Z,Business Finance
787,837322,Essentials of money value: Get a financial Life !,https://www.udemy.com/essentials-of-money-value/,True,20,0,0,20,All Levels,0.616667,2016-05-16T18:28:30Z,Business Finance
778,1035638,Understanding Financial Statements,https://www.udemy.com/understanding-financial-...,True,25,0,0,10,All Levels,1.000000,2016-12-15T14:56:17Z,Business Finance
894,1035638,Understanding Financial Statements,https://www.udemy.com/understanding-financial-...,True,25,0,0,10,All Levels,1.000000,2016-12-15T14:56:17Z,Business Finance
463,1084454,CFA Level 2- Quantitative Methods,https://www.udemy.com/cfa-level-2-quantitative...,True,40,0,0,35,All Levels,5.500000,2017-07-02T14:29:35Z,Business Finance
1100,1084454,CFA Level 2- Quantitative Methods,https://www.udemy.com/cfa-level-2-quantitative...,True,40,0,0,35,All Levels,5.500000,2017-07-02T14:29:35Z,Business Finance


In [42]:
udemy_courses.select_dtypes([object]).replace(np.NaN,'').apply(lambda x: x.str.contains('\n').sum())

course_title           5
url                    0
level                  0
published_timestamp    0
subject                0
dtype: int64

In [43]:
replace_newlines(udemy_courses)

In [44]:
udemy_courses.select_dtypes([object]).replace(np.NaN,'').apply(lambda x: x.str.contains('¦').sum())

course_id              0
course_title           0
url                    0
is_paid                0
price                  0
num_subscribers        0
num_reviews            0
num_lectures           0
level                  0
content_duration       0
published_timestamp    0
subject                0
dtype: int64

In [45]:
udemy_courses['published_timestamp'] = pd.to_datetime(udemy_courses['published_timestamp'], format='%Y-%m-%dT%H:%M:%SZ')
udemy_courses['published_timestamp']

0      2017-01-18 20:58:58
1      2017-03-09 16:34:20
2      2016-12-19 19:26:30
3      2017-05-30 20:07:24
4      2016-12-13 14:57:18
               ...        
3673   2016-06-14 17:36:46
3674   2017-03-10 22:24:30
3675   2015-12-30 16:41:42
3676   2016-08-11 19:06:15
3677   2014-09-28 19:51:11
Name: published_timestamp, Length: 3678, dtype: datetime64[ns]

No hay relevancia en la hora en la que se hizo el lanzamiento de cada curso, por lo que sólo se tomará la fecha.

In [46]:
udemy_courses['published_timestamp'] = udemy_courses['published_timestamp'].dt.date

# Load

In [47]:
coursera_courses.drop_duplicates(inplace=True)
coursera_reviews.drop_duplicates(inplace=True)
edx_courses.drop_duplicates(inplace=True)
udemy_courses.drop_duplicates(inplace=True)

In [48]:
coursera_courses.to_csv('_src/coursera_courses.csv',index=False, sep='¦')
coursera_reviews.to_csv('_src/coursera_reviews.csv',index=False, sep='¦')
edx_courses.to_csv('_src/edx_courses.csv',index=False, sep='¦')
udemy_courses.to_csv('_src/udemy_courses.csv',index=False, sep='¦')